## Setup

In [ ]:
import sys

assert sys.version_info >= (3, 7)

In [ ]:
%pip install transformers "datasets>=1.17.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.5 MB/s eta 0:00:00


In [ ]:
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten,Resizing
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import  Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2
from tensorflow.keras.preprocessing import image

In [ ]:
from transformers import AutoImageProcessor, TFViTModel


In [ ]:
# from tensorflow_addons.metrics import MultiLabelConfusionMatrix
from pathlib import Path
from time import strftime

In [ ]:

def get_run_logdir(root_logdir="my_logs"):
 return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")
run_logdir = get_run_logdir() # e.g., my_logs/run_2022_08_01_17_25_59

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
from PIL import Image

In [ ]:
# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_url="/content/drive/MyDrive/multilabels_data/"
train_csv_file = folder_url+"train_labels.csv"
train_image_path =folder_url+"trainset"

train = pd.read_csv(train_csv_file)
train.head()

,name,HG,HT,TR,CTH,BD,VH,CTQ,DQT,KS,CVN
0,1.jpg,0,1,0,0,0,0,0,0,0,1
1,2.jpg,0,1,0,0,0,0,0,0,0,0
2,3.jpg,0,1,0,0,0,0,1,0,0,0
3,4.jpg,1,0,0,0,0,1,0,0,0,0
4,5.jpg,0,1,0,0,0,1,0,0,0,0


In [ ]:
test_csv_file = folder_url+"test_labels.csv"
test_image_path =folder_url+"testset"

test = pd.read_csv(test_csv_file)
test.head()

,name,HG,HT,TR,CTH,BD,VH,CTQ,DQT,KS,CVN
0,1.jpg,0,1,0,0,0,0,1,0,0,0
1,2.jpg,0,1,0,0,0,0,1,0,0,0
2,3.jpg,0,1,0,0,0,0,1,0,0,0
3,4.jpg,0,1,0,0,0,0,1,0,0,0
4,5.jpg,0,1,0,0,0,0,0,1,0,0


In [ ]:

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size, size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.size = size
        self.indices = np.arange(self.dataset.size) # for shuffle

    def __getitem__(self, i):
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            inds=self.indices[j]
            data.append(self.dataset[inds])

        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        return tuple(batch)
    def on_epoch_end(self):
        np.random.shuffle(self.indices)

    def __len__(self):
        return self.size // self.batch_size

class Dataset:
    def __init__(self,image_path, data, label, w, h,size):
        # the paths of images
        self.data = data
        # the paths of segmentation images
        self.label = label
        self.image_path=image_path
        self.w = w
        self.h = h
        self.size=size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        # read data
        img = image.load_img(self.image_path + '/' + str(self.data[i]),target_size=(IMAGE_W,IMAGE_H,3))
        img = image.img_to_array(img)
        img = img /255
        label = self.label[i]
        return img, label

In [ ]:
y_train = np.array(train.drop(columns=["name"]))
X_train = np.array(train["name"])

y_val = np.array(test.drop(columns=["name"]))
X_val = np.array(test["name"])

In [ ]:
# X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=42, test_size=0.5)

In [ ]:
IMAGE_W = 224
IMAGE_H = 224

In [ ]:
# Xay dung dataset va Dataloader
# Build dataaset
DOUBLE_IMAGE_W,DOUBLE_IMAGE_H=IMAGE_W*2,IMAGE_H*2 #bigger image before augmentation

train_dataset = Dataset(train_image_path,X_train, y_train,DOUBLE_IMAGE_W,DOUBLE_IMAGE_H,len(X_train))
val_dataset = Dataset(test_image_path,X_val, y_val,DOUBLE_IMAGE_W,DOUBLE_IMAGE_H,len(X_val))
# test_dataset = Dataset(test_image_path,X_test, y_test,DOUBLE_IMAGE_W,DOUBLE_IMAGE_H,len(X_test))

# Generator

train_generator = DataGenerator(train_dataset, 16, len(train_dataset))
val_generator = DataGenerator(val_dataset, 16, len(val_dataset))
# test_generator = DataGenerator(test_dataset, 16, len(test_dataset))

In [ ]:

def get_zero_shot_stats(data_generator):
  m = tf.keras.metrics.BinaryAccuracy()
  total_binary_accuracy,total_val_binary_accuracy,total_instance=0,0,0

  for step, (x_batch_train, y_batch_train) in enumerate(data_generator):
      total_instance+=len(y_batch_train)
      for instance in y_batch_train:
          m.update_state(instance, [0]*10) #case full of zero
          total_binary_accuracy+=m.result().numpy()
  return total_binary_accuracy/total_instance

val_zero_shot=get_zero_shot_stats(val_generator)
val_zero_shot

0.813723093830049

In [ ]:

def get_model():
    # model_base_conv = ResNet50(weights='imagenet', include_top=False)
    model_base_conv = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

    # Dong bang cac layer
    # for layer in model_base_conv.layers:
    #     layer.trainable = False
    data_augmentation = tf.keras.Sequential([
          tf.keras.layers.RandomFlip(mode="horizontal_and_vertical", seed=42),
          tf.keras.layers.RandomRotation(factor=0.05, seed=42),
          tf.keras.layers.RandomContrast(factor=0.2, seed=42),
    ])
    # Tao model
    input = Input(shape=(None, None, 3), name='image_input')
    augmented_input=data_augmentation(input)
    resized_image=Resizing(height=IMAGE_H, width=IMAGE_W, crop_to_aspect_ratio=True)(augmented_input)
    permuted_image=tf.keras.layers.Permute((3,1,2))(resized_image)
    output_base_conv = model_base_conv.vit(permuted_image)[0][:,0,:]

    # Them cac layer FC va Dropout
    # x = GlobalAveragePooling2D(name='avg')(output_base_conv)
    x = Dense(2048, activation='relu', name='fc1')(output_base_conv)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation='relu', name='fc2')(x)
    x = Dropout(0.2)(x)
    x = Dense(10, activation='sigmoid', name='predictions')(x)

    # Compile
    my_model = Model(inputs=input, outputs=x)
    # my_model.compile(loss='binary_crossentropy', optimizer='adam',
    #                  metrics=['Accuracy',"BinaryAccuracy","Precision","Recall",
    #                           "TruePositives","TrueNegatives","FalsePositives","FalseNegatives"])

    return my_model

model = get_model()

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [ ]:
del model

In [ ]:
model.save_weights(folder_url+'my_checkpoints/vit_original_check.h5')

In [ ]:
model.load_weights(folder_url+'my_checkpoints/best_weight_check.h5')

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_input (InputLayer)    [(None, None, None, 3)]   0         
                                                                 
 sequential (Sequential)     (None, None, None, 3)     0         
                                                                 
 resizing (Resizing)         (None, 224, 224, 3)       0         
                                                                 
 permute (Permute)           (None, 3, 224, 224)       0         
                                                                 
 vit (TFViTMainLayer)        TFBaseModelOutputWithPoo  86389248  
                             ling(last_hidden_state=(            
                             None, 197, 768),                    
                              pooler_output=(None, 76            
                             8),                             

In [ ]:
def create_accuracy(threshold=0.5):
    def accuracy_fn(y_true, y_pred):
        m=tf.keras.metrics.Accuracy()
        f=lambda x: np.where(x < 0.5, 0, 1)
        y_pred_binary=f(y_pred)
        return np.float32(np.all(y_true==y_pred_binary,axis=1))
    return accuracy_fn
class AccuracyMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=0.5, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        self.accuracy_fn = create_accuracy(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        sample_metrics = self.accuracy_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))
        # print("update",tf.shape(y_true)[0],tf.shape(y_pred)[0],tf.reduce_sum(sample_metrics))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [ ]:
def print_status_bar(step, total, loss, metrics=None):
    step=step+1
    metrics = " - ".join([f"{m.name}: {m.result():.4f}" for m in [loss] + (metrics or [])])
    end = "" if step < total else "\n"
    print(f"\r{step}/{total} - " + metrics)

def print_metrics(epoch,total,loss,val_loss,metrics=None,val_metrics=None):
    metrics = " - ".join([f"{m.name}: {m.result():.4f}" for m in [loss] + (metrics or [])])
    val_metrics = " - val_".join([f"{m.name}: {m.result():.4f}" for m in [val_loss] + (val_metrics or [])])

    end = "" if epoch < total else "\n"
    print(f"\r{epoch}/{total} - " + metrics+" * "+val_metrics)

In [ ]:
n_epochs = 20
batch_size = 16
train_steps = len(train_dataset) // batch_size
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = tf.keras.losses.BinaryCrossentropy()
mean_loss = tf.keras.metrics.Mean(name="mean_loss")
val_mean_loss=tf.keras.metrics.Mean(name="mean_loss")
metrics = [tf.keras.metrics.BinaryCrossentropy(),tf.keras.metrics.BinaryAccuracy(),AccuracyMetric(0.5)]
val_metrics = [tf.keras.metrics.BinaryCrossentropy(),tf.keras.metrics.BinaryAccuracy(),AccuracyMetric(0.5)]

In [ ]:
val_steps = len(val_dataset) // batch_size


In [ ]:
a=np.array([[1, 0, 0],[1,1,1]])
b=np.array([[0.4, 0.1, 0.6],[0.9,0.8,0.9]])
loss_fn(a,b)

<tf.Tensor: shape=(), dtype=float64, numpy=0.39530093395045907>

In [ ]:
import tensorflow as tf

class WeightedCrossEntropy(tf.keras.losses.Loss):
    def __init__(self, weight, epsilon=1e-7, name="weighted_cross_entropy", **kwargs):
        super().__init__(name=name, **kwargs)
        self.weight = weight
        self.epsilon = epsilon
    @tf.function
    def call(self, y_true, y_pred):
        # losses=tf.constant([])
        y_pred = tf.clip_by_value(y_pred, self.epsilon, 1 - self.epsilon)
        return tf.map_fn(lambda t: -tf.reduce_mean(self.weight * (t[0] * tf.math.log(t[1]) + (1 - t[0]) * tf.math.log(1 - t[1]))),(tf.cast(y_true,tf.float32),tf.cast(y_pred,tf.float32)), dtype = tf.float32)
        # for y_true_element,y_pred_element in zip(y_true,y_pred):
        #     # y_pred_element = tf.clip_by_value(y_pred_element, self.epsilon, 1 - self.epsilon)
        #     loss = -tf.reduce_mean(self.weight * y_true_element * tf.math.log(y_pred_element) + (1 - y_true_element) * tf.math.log(1 - y_pred_element))
        #     losses=tf.concat([x, [y]], axis=0)
        # return losses

    def get_config(self):
        config = {
            'weight': self.weight,
            'epsilon': self.epsilon
        }
        base_config = super().get_config()
        return {**base_config, **config}

# wcbe=WeightedCrossEntropy(1)
# def cross_entropy(targets,predictions, epsilon=1e-12):
#     """
#     Computes cross entropy between targets (encoded as one-hot vectors)
#     and predictions.
#     Input: predictions (N, k) ndarray
#            targets (N, k) ndarray
#     Returns: scalar
#     """
#     predictions = np.clip(predictions, epsilon, 1. - epsilon)
#     N = predictions.shape[0]
#     ce = -np.sum(targets*np.log(predictions+1e-9))/N
#     return ce

# def class_weighted_cross_entropy_loss(y_true, y_pred):
#     class_weights=[1,1,1,1,1,1,1,1,1,1]
#     class_weights_tensor = tf.constant(class_weights, dtype=tf.float64)
#     losses=[]
#     for y_true_element,y_pred_element in zip(y_true,y_pred):
#         y_pred_element = tf.clip_by_value(y_pred_element, 1e-7, 1.0 - 1e-7)
#         loss = -tf.reduce_mean(class_weights_tensor * y_true_element * tf.math.log(y_pred_element)+class_weights_tensor*(1-y_true_element)*tf.math.log(1-y_pred_element), axis=-1)
#         losses=np.append(losses,loss)
#     print(losses)
#     return tf.reduce_mean(losses)

# a=tf.constant([[1, 0, 1,0,1,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0]], tf.float32)
# b=tf.constant([[0.4, 0.1,0.2,0.5,0.7,0.8,0.3,0.9,0.2, 0.6],[0.9,0.5,0.4,0.3,0.3,0.4,0.9,0.2,0.8,0.9],[0.9,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]], tf.float32)

# print("wbce",wcbe(a,b))
# print("loss",loss_fn(a,b))

# # print("ce",class_weighted_cross_entropy_loss(a,b))

# losses=[]
# for i in (0,1,2):
#   loss =tf.math.reduce_mean( loss_fn(a[i],b[i]))
#   losses=np.append(losses,loss)
# print("cbe",losses,tf.math.reduce_mean(losses))
# print("natural",loss_fn(a,b))
# loss_fn=WeightedCrossEntropy(1)

In [ ]:
x = tf.constant([1.0, 2.0], tf.float32)
y = tf.constant(9.0, tf.float32)
tf.concat([x, [y]], axis=0)
x


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [ ]:
weight=[1,1,1,1,1,1,1,1,1,1]

In [ ]:
# one_weights = [item * 3 for item in norm_weight]
# zero_weights = [item * 1 for item in norm_weight]


In [ ]:


loss_fn=WeightedCrossEntropy(weight)



In [ ]:
# loss_fn=WeightedCrossEntropy(1)
model.layers[4]

In [ ]:
patience=10
wait=0
best=100000
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(0, train_steps):
        X_batch, y_batch = train_generator.__getitem__(step)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            # print(X_batch,y_batch,y_pred)

            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)

        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step, train_steps, mean_loss, metrics)

    #val
    for step in range(0, val_steps):
        X_val_batch, y_val_batch = val_generator.__getitem__(step)
        y_val_pred = model(X_val_batch)
        val_main_loss = tf.reduce_mean(loss_fn(y_val_batch, y_val_pred))
        val_loss = tf.add_n([val_main_loss]  + model.losses)
        val_mean_loss(val_loss)
        print("val_mean_loss",val_mean_loss.result(),val_loss)
        for metric in val_metrics:
            metric(y_val_batch, y_val_pred)


    print_metrics(epoch,n_epochs,mean_loss,val_mean_loss,metrics,val_metrics)

    wait += 1
    if val_main_loss < best:
      best = val_main_loss
      wait = 0
      model.save_weights(folder_url+'my_checkpoints/best_triple_one_weight_check.h5')
      print("save")
    if wait >= patience:
      break

    for metric in [mean_loss] + metrics + val_metrics:
        metric.reset_states()


Epoch 1/20


1/15 - mean_loss: 0.7108 - binary_crossentropy: 0.7108 - binary_accuracy: 0.4000 - accuracy_metric_7: 0.0000


2/15 - mean_loss: 0.6101 - binary_crossentropy: 0.6101 - binary_accuracy: 0.6281 - accuracy_metric_7: 0.0312


3/15 - mean_loss: 0.5437 - binary_crossentropy: 0.5437 - binary_accuracy: 0.7042 - accuracy_metric_7: 0.0417


4/15 - mean_loss: 0.5742 - binary_crossentropy: 0.5742 - binary_accuracy: 0.7203 - accuracy_metric_7: 0.0312


...

11/15 - mean_loss: 0.4380 - binary_crossentropy: 0.4380 - binary_accuracy: 0.7932 - accuracy_metric_7: 0.0341


12/15 - mean_loss: 0.4373 - binary_crossentropy: 0.4373 - binary_accuracy: 0.7958 - accuracy_metric_7: 0.0365


13/15 - mean_loss: 0.4370 - binary_crossentropy: 0.4370 - binary_accuracy: 0.7966 - accuracy_metric_7: 0.0337


14/15 - mean_loss: 0.4345 - binary_crossentropy: 0.4345 - binary_accuracy: 0.7982 - accuracy_metric_7: 0.0312


15/15 - mean_loss: 0.4356 - binary_crossentropy: 0.4356 - binary_accuracy: 0.7962 - accuracy_metric_7: 0.0292
val_mean_loss tf.Tensor(0.43582925, shape=(), dtype=float32) tf.Tensor(0.44061112, shape=(), dtype=float32)
val_mean_loss tf.Tensor(0.4351536, shape=(), dtype=float32) tf.Tensor(0.4209654, shape=(), dtype=float32)
11/20 - mean_loss: 0.4356 - binary_crossentropy: 0.4356 - binary_accuracy: 0.7962 - accuracy_metric_7: 0.0292 * mean_loss: 0.4352 - val_binary_crossentropy: 0.4308 - val_binary_accuracy: 0.7719 - val_accuracy_metric_8: 0.0000


In [ ]:
model.load_weights(folder_url+'my_checkpoints/best_second_weight_check.h5')

In [ ]:
def evaluate(loss,metrics=None):
    metrics = " - ".join([f"{m.name}: {m.result():.4f}" for m in [loss] + (metrics or [])])

    print( metrics)

In [ ]:
test_steps = len(test_dataset) // batch_size
test_metrics = [tf.keras.metrics.BinaryCrossentropy(),tf.keras.metrics.BinaryAccuracy(),AccuracyMetric(0.5)]
for step in range(0, test_steps):
        X_test_batch, y_test_batch = test_generator.__getitem__(step)
        y_test_pred = model(X_test_batch)
        test_main_loss = tf.reduce_mean(loss_fn(y_test_batch, y_test_pred))
        test_loss = tf.add_n([test_main_loss]  + model.losses)
        mean_loss(test_loss)
        for metric in test_metrics:
            print(y_test_batch.shape,y_test_pred.shape)
            metric(y_test_batch, y_test_pred)


evaluate(mean_loss,test_metrics)

(16, 10) (16, 10)
(16, 10) (16, 10)
(16, 10) (16, 10)
(16, 10) (16, 10)
(16, 10) (16, 10)
(16, 10) (16, 10)
mean_loss: 0.1179 - binary_crossentropy: 0.1786 - binary_accuracy: 0.9406 - accuracy_metric_6: 0.6562


In [ ]:
mean_loss: 0.1638 - binary_crossentropy: 0.1600 - binary_accuracy: 0.9406 - accuracy_metric_12: 0.6250
mean_loss: 0.0601 - binary_crossentropy: 0.2150 - binary_accuracy: 0.9281 - accuracy_metric_15: 0.6562
mean_loss: 0.0356 - binary_crossentropy: 0.1448 - binary_accuracy: 0.9531 - accuracy_metric_14: 0.6875
mean_loss: 0.0601 - binary_crossentropy: 0.2150 - binary_accuracy: 0.9281 - accuracy_metric_15: 0.6562
mean_loss: 0.0529 - binary_crossentropy: 0.1502 - binary_accuracy: 0.9281 - accuracy_metric_19: 0.5938



In [ ]:
X_test_batch, y_test_batch = val_generator.__getitem__(0)

In [ ]:
tf.shape(X_test_batch)
import time

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 16, 224, 224,   3], dtype=int32)>

In [ ]:
import time
start_time = time.time()
y_test_pred = model(X_test_batch)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.9801642894744873 seconds ---


In [ ]:
0.9801642894744873/64

0.015315067023038864

In [ ]:
test_bce=tf.keras.metrics.BinaryCrossentropy()
test_bce(y_test_batch,y_test_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.13443612>

In [ ]:
### binary accuracy
# H_full = history_full.history
H_freeze=history_freeze.history
fig = plt.figure(figsize = (20, 7))
plt.subplot(121)
# plt.plot(H['Accuracy'], label = 'acc')
# plt.plot(H_full['val_binary_accuracy'], label = 'val_bi_acc')
# plt.plot(H_full['binary_accuracy'], label = 'bi_acc')
plt.plot(H_freeze['val_binary_accuracy'], label = 'val_bi_acc_freeze')
plt.plot(H_freeze['binary_accuracy'], label = 'bi_acc_freeze')
plt.axhline(y = val_zero_shot, color = 'r', linestyle = '-',label = 'val_zero_shot')

plt.grid()
plt.legend()
plt.title("binary accuracy")


#### accuracy
plt.subplot(122)
# plt.plot(H['loss'], label = 'loss')
plt.plot(H_freeze['val_Accuracy'], label = 'val_acc_freeze')
plt.plot(H_freeze['Accuracy'], label = 'acc_freeze')

plt.grid()
plt.legend()
plt.title(" accuracy")

In [ ]:
# Dataset hyperparameters
unlabeled_dataset_size = 100000
labeled_dataset_size = 5000
image_size = 96
image_channels = 3

# Algorithm hyperparameters
num_epochs = 20
batch_size = 525  # Corresponds to 200 steps per epoch
width = 128
temperature = 0.1
# Stronger augmentations for contrastive, weaker ones for supervised training
contrastive_augmentation = {"min_area": 0.25, "brightness": 0.6, "jitter": 0.2}
classification_augmentation = {"min_area": 0.75, "brightness": 0.3, "jitter": 0.1}

In [ ]:

# def prepare_dataset():
#     # Labeled and unlabeled samples are loaded synchronously
#     # with batch sizes selected accordingly
#     steps_per_epoch = (unlabeled_dataset_size + labeled_dataset_size) // batch_size
#     unlabeled_batch_size = unlabeled_dataset_size // steps_per_epoch
#     labeled_batch_size = labeled_dataset_size // steps_per_epoch
#     print(
#         f"batch size is {unlabeled_batch_size} (unlabeled) + {labeled_batch_size} (labeled)"
#     )

#     unlabeled_train_dataset = (
#         tfds.load("stl10", split="unlabelled", as_supervised=True, shuffle_files=True)
#         .shuffle(buffer_size=10 * unlabeled_batch_size)
#         .batch(unlabeled_batch_size)
#     )
#     labeled_train_dataset = (
#         tfds.load("stl10", split="train", as_supervised=True, shuffle_files=True)
#         .shuffle(buffer_size=10 * labeled_batch_size)
#         .batch(labeled_batch_size)
#     )
#     test_dataset = (
#         tfds.load("stl10", split="test", as_supervised=True)
#         .batch(batch_size)
#         .prefetch(buffer_size=tf.data.AUTOTUNE)
#     )

#     # Labeled and unlabeled datasets are zipped together
#     train_dataset = tf.data.Dataset.zip(
#         (unlabeled_train_dataset, labeled_train_dataset)
#     ).prefetch(buffer_size=tf.data.AUTOTUNE)

#     return train_dataset, labeled_train_dataset, test_dataset


# # Load STL10 dataset
# train_dataset, labeled_train_dataset, test_dataset = prepare_dataset()

batch size is 500 (unlabeled) + 25 (labeled)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Generating test examples...:   0%|          | 0/8000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stl10/1.0.0.incompleteYD0949/stl10-test.tfrecord*...:   0%|          | 0/8…

Generating unlabelled examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stl10/1.0.0.incompleteYD0949/stl10-unlabelled.tfrecord*...:   0%|         …

Dataset stl10 downloaded and prepared to /root/tensorflow_datasets/stl10/1.0.0. Subsequent calls will reuse this data.


## Image augmentations

The two most important image augmentations for contrastive learning are the
following:

- Cropping: forces the model to encode different parts of the same image
similarly, we implement it with the
[RandomTranslation](https://keras.io/api/layers/preprocessing_layers/image_preprocessing/random_translation/)
and
[RandomZoom](https://keras.io/api/layers/preprocessing_layers/image_preprocessing/random_zoom/)
layers
- Color jitter: prevents a trivial color histogram-based solution to the task by
distorting color histograms. A principled way to implement that is by affine
transformations in color space.

In this example we use random horizontal flips as well. Stronger augmentations
are applied for contrastive learning, along with weaker ones for supervised
classification to avoid overfitting on the few labeled examples.

We implement random color jitter as a custom preprocessing layer. Using
preprocessing layers for data augmentation has the following two advantages:

- The data augmentation will run on GPU in batches, so the training will not be
bottlenecked by the data pipeline in environments with constrained CPU
resources (such as a Colab Notebook, or a personal machine)
- Deployment is easier as the data preprocessing pipeline is encapsulated in the
model, and does not have to be reimplemented when deploying it

In [ ]:

# Distorts the color distibutions of images
class RandomColorAffine(layers.Layer):
    def __init__(self, brightness=0, jitter=0, **kwargs):
        super().__init__(**kwargs)

        self.brightness = brightness
        self.jitter = jitter

    def get_config(self):
        config = super().get_config()
        config.update({"brightness": self.brightness, "jitter": self.jitter})
        return config

    def call(self, images, training=True):
        if training:
            batch_size = tf.shape(images)[0]

            # Same for all colors
            brightness_scales = 1 + tf.random.uniform(
                (batch_size, 1, 1, 1), minval=-self.brightness, maxval=self.brightness
            )
            # Different for all colors
            jitter_matrices = tf.random.uniform(
                (batch_size, 1, 3, 3), minval=-self.jitter, maxval=self.jitter
            )

            color_transforms = (
                tf.eye(3, batch_shape=[batch_size, 1]) * brightness_scales
                + jitter_matrices
            )
            images = tf.clip_by_value(tf.matmul(images, color_transforms), 0, 1)
        return images


# Image augmentation module
def get_augmenter(min_area, brightness, jitter):
    zoom_factor = 1.0 - math.sqrt(min_area)
    return keras.Sequential(
        [
            keras.Input(shape=(image_size, image_size, image_channels)),
            layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),
            layers.Rescaling(1 / 255),
            layers.RandomFlip("horizontal"),
            layers.RandomTranslation(zoom_factor / 2, zoom_factor / 2),
            layers.RandomZoom((-zoom_factor, 0.0), (-zoom_factor, 0.0)),
            RandomColorAffine(brightness, jitter),
        ]
    )


def visualize_augmentations(num_images):
    # Sample a batch from a dataset
    images = next(iter(train_dataset))[0][0][:num_images]
    # Apply augmentations
    augmented_images = zip(
        images,
        get_augmenter(**classification_augmentation)(images),
        get_augmenter(**contrastive_augmentation)(images),
        get_augmenter(**contrastive_augmentation)(images),
    )
    row_titles = [
        "Original:",
        "Weakly augmented:",
        "Strongly augmented:",
        "Strongly augmented:",
    ]
    plt.figure(figsize=(num_images * 2.2, 4 * 2.2), dpi=100)
    for column, image_row in enumerate(augmented_images):
        for row, image in enumerate(image_row):
            plt.subplot(4, num_images, row * num_images + column + 1)
            plt.imshow(image)
            if column == 0:
                plt.title(row_titles[row], loc="left")
            plt.axis("off")
    plt.tight_layout()


visualize_augmentations(num_images=8)

NameError: ignored

## Encoder architecture

In [ ]:

# Define the encoder architecture
def get_encoder():
    # base_model=ResNet50(weights='imagenet', include_top=False)
    # for layer in base_model.layers:
    #       layer.trainable = False
    return keras.Sequential(
        [
            keras.Input(shape=(224, 224, image_channels)),
            # layers.Lambda(tf.keras.applications.resnet50.preprocess_input),
            # base_model,
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Conv2D(width, kernel_size=3, strides=2, activation="relu"),
            layers.Flatten(),
            layers.Dense(width, activation="relu"),
        ],
        name="encoder",
    )


## Supervised baseline model

A baseline supervised model is trained using random initialization.

In [ ]:
# Baseline supervised training with random initialization
baseline_model = keras.Sequential(
    [
        keras.Input(shape=(image_size, image_size, image_channels)),
        get_augmenter(**classification_augmentation),
        get_encoder(),
        layers.Dense(10),
    ],
    name="baseline_model",
)


In [ ]:
# baseline_model.summary()
baseline_model.layers[1].summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 111, 111, 128)     3584      
                                                                 
 conv2d_1 (Conv2D)           (None, 55, 55, 128)       147584    
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 128)       147584    
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 128)       147584    
                                                                 
 flatten (Flatten)           (None, 21632)             0         
                                                                 
 dense (Dense)               (None, 128)               2769024   
                                                                 
Total params: 3,215,360
Trainable params: 3,215,360
Non-tra

In [ ]:
baseline_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

baseline_history = baseline_model.fit(
    labeled_train_dataset, epochs=num_epochs, validation_data=test_dataset
)

print(
    "Maximal validation accuracy: {:.2f}%".format(
        max(baseline_history.history["val_acc"]) * 100
    )
)

Epoch 1/20
200/200 [==============================] - 71s 297ms/step - loss: 2.2922 - acc: 0.1284 - val_loss: 2.0218 - val_acc: 0.2004
Epoch 2/20
200/200 [==============================] - 40s 201ms/step - loss: 2.0110 - acc: 0.1982 - val_loss: 1.9075 - val_acc: 0.2210
Epoch 3/20
200/200 [==============================] - 41s 205ms/step - loss: 1.9617 - acc: 0.2098 - val_loss: 1.9304 - val_acc: 0.2192
Epoch 4/20
200/200 [==============================] - 42s 208ms/step - loss: 1.9514 - acc: 0.2202 - val_loss: 1.9027 - val_acc: 0.2346
Epoch 5/20
200/200 [==============================] - 42s 210ms/step - loss: 1.9071 - acc: 0.2340 - val_loss: 1.8114 - val_acc: 0.2539
Epoch 6/20
200/200 [==============================] - 43s 213ms/step - loss: 1.8912 - acc: 0.2324 - val_loss: 1.7919 - val_acc: 0.2640
Epoch 7/20
200/200 [==============================] - 43s 214ms/step - loss: 1.9008 - acc: 0.2404 - val_loss: 1.7988 - val_acc: 0.2769
Epoch 8/20
200/200 [==============================] - 4

In [ ]:
for layer in baseline_model.layers[1].layers[1].layers[150:]:
          layer.trainable = True
baseline_model.summary()

Model: "baseline_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 encoder (Sequential)        (None, 128)               26276352  
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 26,277,642
Trainable params: 12,680,074
Non-trainable params: 13,597,568
_________________________________________________________________


In [ ]:
baseline_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

baseline_history = baseline_model.fit(
    labeled_train_dataset, epochs=num_epochs, validation_data=test_dataset
)

print(
    "Maximal validation accuracy: {:.2f}%".format(
        max(baseline_history.history["val_acc"]) * 100
    )
)

Epoch 1/20
200/200 [==============================] - 75s 323ms/step - loss: 1.4387 - acc: 0.4560 - val_loss: 2.7004 - val_acc: 0.2848
Epoch 2/20
200/200 [==============================] - 45s 223ms/step - loss: 1.3862 - acc: 0.4814 - val_loss: 1.3544 - val_acc: 0.4981
Epoch 3/20
200/200 [==============================] - 45s 227ms/step - loss: 1.3696 - acc: 0.4836 - val_loss: 1.4032 - val_acc: 0.4616


KeyboardInterrupt: ignored

## Self-supervised model for contrastive pretraining

We pretrain an encoder on unlabeled images with a contrastive loss.
A nonlinear projection head is attached to the top of the encoder, as it
improves the quality of representations of the encoder.

We use the InfoNCE/NT-Xent/N-pairs loss, which can be interpreted in the
following way:

1. We treat each image in the batch as if it had its own class.
2. Then, we have two examples (a pair of augmented views) for each "class".
3. Each view's representation is compared to every possible pair's one (for both
  augmented versions).
4. We use the temperature-scaled cosine similarity of compared representations as
  logits.
5. Finally, we use categorical cross-entropy as the "classification" loss

The following two metrics are used for monitoring the pretraining performance:

- [Contrastive accuracy (SimCLR Table 5)](https://arxiv.org/abs/2002.05709):
Self-supervised metric, the ratio of cases in which the representation of an
image is more similar to its differently augmented version's one, than to the
representation of any other image in the current batch. Self-supervised
metrics can be used for hyperparameter tuning even in the case when there are
no labeled examples.
- [Linear probing accuracy](https://arxiv.org/abs/1603.08511): Linear probing is
a popular metric to evaluate self-supervised classifiers. It is computed as
the accuracy of a logistic regression classifier trained on top of the
encoder's features. In our case, this is done by training a single dense layer
on top of the frozen encoder. Note that contrary to traditional approach where
the classifier is trained after the pretraining phase, in this example we
train it during pretraining. This might slightly decrease its accuracy, but
that way we can monitor its value during training, which helps with
experimentation and debugging.

Another widely used supervised metric is the
[KNN accuracy](https://arxiv.org/abs/1805.01978), which is the accuracy of a KNN
classifier trained on top of the encoder's features, which is not implemented in
this example.

In [ ]:

# Define the contrastive model with model-subclassing
class ContrastiveModel(keras.Model):
    def __init__(self):
        super().__init__()

        self.temperature = temperature
        self.contrastive_augmenter = get_augmenter(**contrastive_augmentation)
        self.classification_augmenter = get_augmenter(**classification_augmentation)
        self.encoder = get_encoder()
        # Non-linear MLP as projection head
        self.projection_head = keras.Sequential(
            [
                keras.Input(shape=(width,)),
                layers.Dense(width, activation="relu"),
                layers.Dense(width),
            ],
            name="projection_head",
        )
        # Single dense layer for linear probing
        self.linear_probe = keras.Sequential(
            [layers.Input(shape=(width,)), layers.Dense(10)], name="linear_probe"
        )

        self.encoder.summary()
        self.projection_head.summary()
        self.linear_probe.summary()

    def compile(self, contrastive_optimizer, probe_optimizer, **kwargs):
        super().compile(**kwargs)

        self.contrastive_optimizer = contrastive_optimizer
        self.probe_optimizer = probe_optimizer

        # self.contrastive_loss will be defined as a method
        self.probe_loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

        self.contrastive_loss_tracker = keras.metrics.Mean(name="c_loss")
        self.contrastive_accuracy = keras.metrics.SparseCategoricalAccuracy(
            name="c_acc"
        )
        self.probe_loss_tracker = keras.metrics.Mean(name="p_loss")
        self.probe_accuracy = keras.metrics.SparseCategoricalAccuracy(name="p_acc")

    @property
    def metrics(self):
        return [
            self.contrastive_loss_tracker,
            self.contrastive_accuracy,
            self.probe_loss_tracker,
            self.probe_accuracy,
        ]

    def contrastive_loss(self, projections_1, projections_2):
        # InfoNCE loss (information noise-contrastive estimation)
        # NT-Xent loss (normalized temperature-scaled cross entropy)

        # Cosine similarity: the dot product of the l2-normalized feature vectors
        projections_1 = tf.math.l2_normalize(projections_1, axis=1)
        projections_2 = tf.math.l2_normalize(projections_2, axis=1)
        similarities = (
            tf.matmul(projections_1, projections_2, transpose_b=True) / self.temperature
        )

        # The similarity between the representations of two augmented views of the
        # same image should be higher than their similarity with other views
        batch_size = tf.shape(projections_1)[0]
        contrastive_labels = tf.range(batch_size)
        self.contrastive_accuracy.update_state(contrastive_labels, similarities)
        self.contrastive_accuracy.update_state(
            contrastive_labels, tf.transpose(similarities)
        )

        # The temperature-scaled similarities are used as logits for cross-entropy
        # a symmetrized version of the loss is used here
        loss_1_2 = keras.losses.sparse_categorical_crossentropy(
            contrastive_labels, similarities, from_logits=True
        )
        loss_2_1 = keras.losses.sparse_categorical_crossentropy(
            contrastive_labels, tf.transpose(similarities), from_logits=True
        )
        return (loss_1_2 + loss_2_1) / 2

    def train_step(self, data):
        (unlabeled_images, _), (labeled_images, labels) = data

        # Both labeled and unlabeled images are used, without labels
        images = tf.concat((unlabeled_images, labeled_images), axis=0)
        # Each image is augmented twice, differently
        augmented_images_1 = self.contrastive_augmenter(images, training=True)
        augmented_images_2 = self.contrastive_augmenter(images, training=True)
        with tf.GradientTape() as tape:
            features_1 = self.encoder(augmented_images_1, training=True)
            features_2 = self.encoder(augmented_images_2, training=True)
            # The representations are passed through a projection mlp
            projections_1 = self.projection_head(features_1, training=True)
            projections_2 = self.projection_head(features_2, training=True)
            contrastive_loss = self.contrastive_loss(projections_1, projections_2)
        gradients = tape.gradient(
            contrastive_loss,
            self.encoder.trainable_weights + self.projection_head.trainable_weights,
        )
        self.contrastive_optimizer.apply_gradients(
            zip(
                gradients,
                self.encoder.trainable_weights + self.projection_head.trainable_weights,
            )
        )
        self.contrastive_loss_tracker.update_state(contrastive_loss)

        # Labels are only used in evalutation for an on-the-fly logistic regression
        preprocessed_images = self.classification_augmenter(
            labeled_images, training=True
        )
        with tf.GradientTape() as tape:
            # the encoder is used in inference mode here to avoid regularization
            # and updating the batch normalization paramers if they are used
            features = self.encoder(preprocessed_images, training=False)
            class_logits = self.linear_probe(features, training=True)
            probe_loss = self.probe_loss(labels, class_logits)
        gradients = tape.gradient(probe_loss, self.linear_probe.trainable_weights)
        self.probe_optimizer.apply_gradients(
            zip(gradients, self.linear_probe.trainable_weights)
        )
        self.probe_loss_tracker.update_state(probe_loss)
        self.probe_accuracy.update_state(labels, class_logits)

        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        labeled_images, labels = data

        # For testing the components are used with a training=False flag
        preprocessed_images = self.classification_augmenter(
            labeled_images, training=False
        )
        features = self.encoder(preprocessed_images, training=False)
        class_logits = self.linear_probe(features, training=False)
        probe_loss = self.probe_loss(labels, class_logits)
        self.probe_loss_tracker.update_state(probe_loss)
        self.probe_accuracy.update_state(labels, class_logits)

        # Only the probe metrics are logged at test time
        return {m.name: m.result() for m in self.metrics[2:]}


# Contrastive pretraining
pretraining_model = ContrastiveModel()
pretraining_model.compile(
    contrastive_optimizer=keras.optimizers.Adam(),
    probe_optimizer=keras.optimizers.Adam(),
)

pretraining_history = pretraining_model.fit(
    train_dataset, epochs=num_epochs, validation_data=test_dataset
)
print(
    "Maximal validation accuracy: {:.2f}%".format(
        max(pretraining_history.history["val_p_acc"]) * 100
    )
)

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_2 (Lambda)           (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 avg (GlobalAveragePooling2D  (None, 2048)             0         
 )                                                               
                                                                 
 fc1 (Dense)                 (None, 1024)              2098176   
                                                                 
 dropout_4 (Dropout)         (None, 1024)              0         
                                                                 
 fc2 (Dense)                 (None, 512)               524800    
                                                           

KeyboardInterrupt: ignored

## Supervised finetuning of the pretrained encoder

We then finetune the encoder on the labeled examples, by attaching
a single randomly initalized fully connected classification layer on its top.

In [ ]:
# Supervised finetuning of the pretrained encoder
finetuning_model = keras.Sequential(
    [
        layers.Input(shape=(image_size, image_size, image_channels)),
        get_augmenter(**classification_augmentation),
        pretraining_model.encoder,
        layers.Dense(10),
    ],
    name="finetuning_model",
)
finetuning_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

finetuning_history = finetuning_model.fit(
    labeled_train_dataset, epochs=num_epochs, validation_data=test_dataset
)
print(
    "Maximal validation accuracy: {:.2f}%".format(
        max(finetuning_history.history["val_acc"]) * 100
    )
)

## Comparison against the baseline

In [ ]:

# The classification accuracies of the baseline and the pretraining + finetuning process:
def plot_training_curves(pretraining_history, finetuning_history, baseline_history):
    for metric_key, metric_name in zip(["acc", "loss"], ["accuracy", "loss"]):
        plt.figure(figsize=(8, 5), dpi=100)
        plt.plot(
            baseline_history.history[f"val_{metric_key}"], label="supervised baseline"
        )
        plt.plot(
            pretraining_history.history[f"val_p_{metric_key}"],
            label="self-supervised pretraining",
        )
        plt.plot(
            finetuning_history.history[f"val_{metric_key}"],
            label="supervised finetuning",
        )
        plt.legend()
        plt.title(f"Classification {metric_name} during training")
        plt.xlabel("epochs")
        plt.ylabel(f"validation {metric_name}")


plot_training_curves(pretraining_history, finetuning_history, baseline_history)